<a href="https://colab.research.google.com/github/usamasansi/Chatbot/blob/main/google_gemma_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install gradio
!pip install micropip

In [6]:
import torch
import gradio as gr
import torchaudio
import torchaudio.transforms as transforms
import soundfile as sf
import tempfile


try:
    from transformers import AutoModelForCausalLM, AutoTokenizer
except ModuleNotFoundError:
    raise ImportError("The 'transformers' library is not installed. Please install it using 'pip install transformers'.")

## Ensure all required dependencies are installed
missing_dependencies = []
optional_dependencies = ["micropip"]
for dep in optional_dependencies:
    try:
        __import__(dep)
    except ModuleNotFoundError:
        missing_dependencies.append(dep)

if missing_dependencies:
    print(f"Warning: Missing optional dependencies: {', '.join(missing_dependencies)}. Ensure all required packages are installed.")


In [ ]:
## Function to load a chatbot model
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto"
    )
    return model, tokenizer

## Load multiple chatbot models
models = {}
try:
    models = {



        "Falcon 7B": load_model("tiiuae/falcon-7b")
    }
except Exception as e:
    print(f"Error loading models: {e}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
## Chatbot response function
def chat(model_name, user_input):
    if model_name not in models:
        return "Model not available. Please select a different model."
    model, tokenizer = models[model_name]
    inputs = tokenizer(user_input, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    output = model.generate(**inputs, max_length=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [2]:
## Voice-to-Text function
def speech_to_text(audio):
    try:
        waveform, sample_rate = torchaudio.load(audio)
        transform = transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        resampled_waveform = transform(waveform)
        return "[Voice recognition is in progress...]"
    except Exception as e:
        return f"Error processing audio: {e}"


In [3]:
## Text-to-Speech function
def text_to_speech(text):
    try:
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        sf.write(temp_file.name, [0], 22050)  # Placeholder for TTS processing
        return temp_file.name
    except Exception as e:
        return f"Error generating speech: {e}"


In [4]:






## Web GUI using Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Multi-Model AI Chatbot with Voice Support")

    chatbot_model = gr.Radio(choices=list(models.keys()), label="Select Chatbot Model")
    user_input = gr.Textbox(label="Enter your message")
    response_output = gr.Textbox(label="Chatbot Response")
    chat_button = gr.Button("Chat")

    chat_button.click(chat, inputs=[chatbot_model, user_input], outputs=response_output)

    gr.Markdown("### Voice Chat Support")
    audio_input = gr.Audio(source="microphone", type="filepath")
    voice_output = gr.Textbox(label="Transcribed Text")
    speech_button = gr.Button("Convert Speech to Text")

    speech_button.click(speech_to_text, inputs=[audio_input], outputs=[voice_output])

    gr.Markdown("### Text-to-Speech")
    tts_input = gr.Textbox(label="Enter text for speech output")
    tts_output = gr.Audio(label="Generated Speech Output")
    tts_button = gr.Button("Generate Speech")

    tts_button.click(text_to_speech, inputs=[tts_input], outputs=[tts_output])

demo.launch()


NameError: name 'gr' is not defined